# Introduction

### Scenario

A couple originally from France living and working in a Small Dutch city called Wageningen. However, at some point, they were thought to move back in there a home city in France, Lyon. Wageningen is a relatively a small city in comparison to Lyon. Most of the amenities such as library, hospital, markets, are located near to the city center. On the other hand, Lyon has huge infrastructure and sometimes it is hard to find a good neighborhood that satisfy all needs of old age people. For example, this french couple is looking forward to having the following facilities in their nearby neighborhood

* Local library
* Health centre
* Grocery store 
* Social centre

In order to meet their requirement these couple is looking for a company that can assist in their relocation.  

### Business case 

A fictitious company called RELOCATE ME facilitate retired people to relocate in their preferred city and in return company earn revenue. The company proposes the best relocation package considering the specific requirement such as social engagement, partial jobs, medical visits, etc., for retired people. Often, finding such recommendations are not easy and require a detailed analysis of the particular city and neighborhood. 

### Role of data science

RELOCATE ME uses the power of data science to explore cities, taking into account the various needs of retired or old age people. RELOCATE ME uses data from different public sources to find, the better the recommendations for their client.

### Data used for problem solving 

The following data sets are used to in this case study:
* Foursquare data to find and compare locations
* Public data set about the city Lyon in France 
  https://data.beta.grandlyon.com/en/accueil

### Code below is just for refrenec of the City Wageningen in Netehrland

In [3]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [4]:

address = 'Bowlespark, Wageningen'

geolocator = Nominatim(user_agent="Wageningen_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('31 Bowlespark, Wageningen are {}, {}.'.format(latitude, longitude))





31 Bowlespark, Wageningen are 51.9649102, 5.6674073.


In [5]:
CLIENT_ID = '-----' # your Foursquare ID
CLIENT_SECRET = '----' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U
CLIENT_SECRET:15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK


In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
#latitude=51.9649102
#longitude=5.6674073

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=X52HWHL32ARKRE4ZGHER5IMTAMCCMT2SBXVYCYE2DSVQ1Z5U&client_secret=15R0HBHZ1TWTCB1EDA4BNTPMLDLBRVVO0FKGLP0DBGSPDPTK&v=20180605&ll=51.9649102,5.6674073&radius=1000&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3f6b577ca06c0023680256'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wageningen',
  'headerFullLocation': 'Wageningen',
  'headerLocationGranularity': 'city',
  'totalResults': 50,
  'suggestedBounds': {'ne': {'lat': 51.97391020900001,
    'lng': 5.681987017548056},
   'sw': {'lat': 51.95591019099999, 'lng': 5.652827582451944}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba3d7aaf964a520956538e3',
       'name': 'Cafe De Zaaier',
       'location': {'address': 'Herenstraat 33',
        'lat': 51.96616579368193,
        'lng': 5.664414799773752,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.96616579368193,
      

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(100)

,name,categories,lat,lng
0,Cafe De Zaaier,Pub,51.966166,5.664415
1,Restaurant O Mundo,French Restaurant,51.967622,5.667527
2,Heerenstraat Theater,Movie Theater,51.966807,5.665229
3,IJssalon Cicuto,Ice Cream Shop,51.967242,5.663982
4,De Vlaamsche Reus,Beer Garden,51.965141,5.662214
5,Markt,Plaza,51.965096,5.662558
6,Cafe The Doctor,Pub,51.964688,5.662559
7,My Asia,Asian Restaurant,51.964875,5.661632
8,Le Perron,Sandwich Place,51.965665,5.662680
9,Bagels & Beans,Bagel Shop,51.965063,5.662269


In [12]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg